In [1]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas as pd
from backtesting.test import SMA
import pandas as pd
from bokeh.io import export_png
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import Position

/Users/fahad/anaconda3/lib/python3.7/site-packages/backtesting/_plotting.py:34: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `bokeh.io.reset_output()`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
GOOG_predictions = pd.read_csv("/Users/fahad/project_repo/models/final_models/Financial_Analysis/predictions/GOOG_predictions.csv", index_col=0)


In [3]:
GOOG_predictions.head()

Date  Predictions
0  2019-07-31            0
1  2019-06-28            0
2  2019-05-31            1
3  2019-04-30            0
4  2019-03-29            0

In [4]:
GOOG_predictions.Date = pd.to_datetime(GOOG_predictions.Date, format="%Y-%m-%d")

In [5]:
GOOG_predictions['Predictions'] = GOOG_predictions['Predictions'].shift(-1)


In [6]:
GOOG_predictions = GOOG_predictions.dropna()


In [7]:
GOOG_predictions = GOOG_predictions[::-1]

In [8]:
GOOG_predictions.head(10)

Date  Predictions
112 2010-03-31          0.0
111 2010-04-30          0.0
110 2010-05-28          0.0
109 2010-06-30          1.0
108 2010-07-30          1.0
107 2010-08-31          1.0
106 2010-09-30          1.0
105 2010-10-29          1.0
104 2010-11-30          1.0
103 2010-12-31          1.0

In [9]:
GOOG_monthly = pd.read_csv("/Users/fahad/project_repo/data/final/GOOG/GOOG_monthly.csv", index_col =0)

In [10]:
GOOG_monthly.columns = ['Open','High','Low','Close','Volume']

In [11]:
GOOG_predictions = GOOG_predictions.set_index("Date")

In [12]:
GOOG_monthly.head()

Open     High        Low    Close    Volume
2019-08-15  1213.9711  1234.11  1140.1400  1167.26  16723300
2019-07-31  1098.0000  1265.55  1093.7666  1216.68  32949200
2019-06-28  1065.5000  1124.11  1025.0000  1080.91  34096800
2019-05-31  1188.0500  1190.85  1100.1800  1103.63  33317400
2019-04-30  1184.1000  1289.27  1175.0000  1188.48  27855200

In [13]:
GOOG_final_df = pd.merge(GOOG_predictions, GOOG_monthly, right_index=True, left_index=True)

In [14]:
GOOG_final_df.index = pd.to_datetime(GOOG_final_df.index, format ="%Y-%m-%d")

In [15]:
equity_dict = dict()

In [16]:
class BH(TrailingStrategy):

    
    def init(self):
        
        super().init()
        self.set_trailing_sl(4)
        self.start = self.data.index[0]
        
    def next(self):
        super().init()
        equity_dict[self.data.index[-1]] = TrailingStrategy.equity.__get__(self)
        
        position = Strategy.position.__get__(self)
#         print (type(self.start))
        if self.data.index[-1] > self.start and (position.is_long!=True):
        
            self.buy()
        else:
            pass

In [17]:
bt = Backtest(GOOG_final_df, BH, cash=10000, commission=.01)
bt.run()



Start                     2010-03-31 00:00:00
End                       2019-07-31 00:00:00
Duration                   3409 days 00:00:00
Exposure [%]                                0
Equity Final [$]                      45931.6
Equity Peak [$]                       45988.6
Return [%]                            359.316
Buy & Hold Return [%]                 330.682
Max. Drawdown [%]                    -17.4874
Avg. Drawdown [%]                     -8.2595
Max. Drawdown Duration      518 days 00:00:00
Avg. Drawdown Duration      188 days 00:00:00
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                            NaN
Max. Trade Duration                       NaT
Avg. Trade Duration                       NaT
Expectancy [%]                            NaN
SQN                                       NaN
Sharpe Ratio                      

In [18]:
bt.plot(superimpose=False, plot_volume=False)

In [19]:
df = pd.DataFrame.from_dict([equity_dict])

In [20]:
df = df.T

In [21]:
df.columns = ['Equity']

In [22]:
export = df.to_csv("/Users/fahad/project_repo/data/final/Equity/BH/GOOG_BH_Equity.csv")

In [23]:
df

Equity
2010-04-30  10000.000000
2010-05-28   9132.417282
2010-06-30   8367.415762
2010-07-30   9117.747023
2010-08-31   8462.761119
2010-09-30   9887.637370
2010-10-29  11540.808670
2010-11-30  10450.290493
2010-12-31  11169.782499
2011-01-31  11289.949510
2011-02-28  11535.168585
2011-03-31  11034.197747
2011-04-29  10231.961858
2011-05-31   9948.375865
2011-06-30   9522.625023
2011-07-29  11352.568033
2011-08-31  10172.914633
2011-09-30   9685.477731
2011-10-31  11144.772086
2011-11-30  11271.707953
2011-12-30  12146.340021
2012-01-31  10909.141908
2012-02-29  11626.372595
2012-03-30  12058.707310
2012-04-30  11374.384664
2012-05-31  10923.245893
2012-06-29  10908.386877
2012-07-31  11903.189732
2012-08-31  12883.318553
2012-09-28  14188.594066
...                  ...
2017-02-28  31077.467083
2017-03-31  31317.189530
2017-04-28  34201.409213
2017-05-31  36424.976482
2017-06-30  34305.981052
2017-07-31  35127.832656
2017-08-31  35461.178988
2017-09-29  36207.904974
2017-10-31  38379.752597
2017-11-30  38559.827569
2017-12-29  39503.239217
2018-01-31  44167.067746
2018-02-28  41705.288092
2018-03-29  38951.688830
2018-04-30  38405.801178
2018-05-31  40960.072169
2018-06-29  42117.535199
2018-07-31  45953.471874
2018-08-31  45988.580831
2018-09-28  45055.362106
2018-10-31  40649.754292
2018-11-30  41316.446957
2018-12-31  39095.899814
2019-01-31  42144.716327
2019-02-28  42278.734388
2019-03-29  44294.290525
2019-04-30  44866.981789
2019-05-31  41663.761369
2019-06-28  40806.045777
2019-07-31  45931.575965

[112 rows x 1 columns]